In [1]:
import numpy as np
import pymc3 as pm

from aesara import tensor as at

import aesara, warnings

from aesara.tensor.random.op import RandomVariable

from pymc3.distributions.continuous import assert_negative_support
from pymc3.distributions.distribution import Continuous

You are running the v4 development version of PyMC3 which currently still lacks key features. You probably want to use the stable v3 instead which you can either install via conda or find on the v3 GitHub branch: https://github.com/pymc-devs/pymc3/tree/v3


In [2]:
class StickBreakingWeightsRV(RandomVariable):
    name = "stick_breaking_weights"
    ndim_supp = 1
    ndims_params = [0]
    dtype = "floatX"
    _print_name = ("Stick-Breaking W=eights", "\\operatorname{StickBreakingWeights}")

    def __call__(self, alpha, size=None, **kwargs):
        return super().__call__(alpha, size=size, **kwargs)

    @classmethod
    def rng_fn(cls, rng, alpha, size):
        betas = rng.beta(1, alpha, size=size)
        sticks = np.concatenate(
            [
                [1],
                np.cumprod(1 - betas[:-1]),
            ]
        )


        return betas * sticks


stickbreakingweights = StickBreakingWeightsRV()


class StickBreakingWeights(Continuous):
    rv_op = stickbreakingweights

    @classmethod
    def dist(cls, alpha, *args, **kwargs):
        alpha = at.as_tensor_variable(alpha)

        assert_negative_support(alpha, "alpha", "StickBreakingWeights")

        return super().dist([alpha], **kwargs)

    def logp(value, alpha):
        # alpha=1, beta=alpha is confusing... need to revisit this
        return bound(
            at.sum(pm.Beta.logp(value, alpha=1, beta=alpha)),
            alpha > 0,
        )

    def _distr_parameters_for_repr(self):
        return ["alpha"]

In [3]:
with pm.Model() as model:
    sbw = StickBreakingWeights("test-sticks", alpha=1)
    # larry_norm = LarryNormal(name="larrynormal", mu=1, sigma=2)

    trace = pm.sample(1000)
    print(trace.to_dict()["posterior"]["test-sticks"][0].mean())

IndexError: tuple index out of range

In [4]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Mon Jul 12 2021

Python implementation: CPython
Python version       : 3.8.10
IPython version      : 7.25.0

numpy : 1.21.0
pymc3 : 4.0
aesara: 2.0.12

Watermark: 2.2.0

